In [3]:
# tensor格式的分割标签下采样测试
import torch
import torch.nn.functional as F

img_size = 8
target_size = 2

label = torch.randint(0, 10, (2, 1, img_size, img_size), dtype=torch.float32)  # 示例标签
# label_subsample = F.interpolate(label, scale_factor=target_size/img_size, mode='nearest')
label_subsample = F.interpolate(label, (target_size, target_size), mode='nearest')


print(label_subsample.shape)
print(label)
print(label_subsample)

torch.Size([2, 1, 2, 2])
tensor([[[[9., 4., 7., 7., 7., 5., 8., 8.],
          [4., 1., 4., 4., 2., 6., 7., 4.],
          [2., 8., 0., 2., 9., 9., 1., 7.],
          [7., 7., 4., 8., 2., 0., 2., 6.],
          [6., 4., 0., 9., 8., 1., 7., 8.],
          [4., 8., 3., 1., 0., 4., 9., 2.],
          [7., 9., 5., 6., 5., 8., 3., 5.],
          [5., 9., 1., 2., 4., 7., 7., 7.]]],


        [[[1., 7., 0., 2., 2., 9., 3., 7.],
          [1., 5., 1., 4., 1., 6., 8., 0.],
          [5., 9., 2., 0., 6., 1., 0., 1.],
          [8., 2., 6., 5., 2., 1., 3., 7.],
          [3., 3., 2., 5., 3., 3., 6., 7.],
          [1., 6., 6., 5., 3., 4., 0., 2.],
          [4., 8., 0., 6., 6., 0., 9., 2.],
          [3., 7., 4., 1., 5., 3., 0., 7.]]]])
tensor([[[[9., 7.],
          [6., 8.]]],


        [[[1., 2.],
          [3., 3.]]]])


In [10]:
# affinity交叉熵损失计算测试
import torch
import torch.nn as nn
import torch.nn.functional as F

target_size = 4

func = nn.ReLU()

affinity = torch.randn((target_size, target_size), dtype=torch.float32)
affinity = func(affinity)
affinity = F.softmax(affinity, dim=-1)
label = torch.randn((target_size, target_size), dtype=torch.float32)
label = func(label)
label = F.softmax(label, dim=-1)

loss = (-label*affinity.log()).mean()
print(affinity)
print(label)
print(loss)

tensor([[0.0961, 0.6014, 0.2064, 0.0961],
        [0.1942, 0.1942, 0.4174, 0.1942],
        [0.1877, 0.1877, 0.2986, 0.3260],
        [0.4954, 0.1682, 0.1682, 0.1682]])
tensor([[0.0895, 0.1266, 0.2678, 0.5161],
        [0.6959, 0.1537, 0.0752, 0.0752],
        [0.2301, 0.2301, 0.2301, 0.3096],
        [0.1692, 0.1692, 0.1837, 0.4779]])
tensor(0.4051)


In [172]:
# KL散度损失与交叉熵损失对比

import torch
import torch.nn as nn
import torch.nn.functional as F

# 假设我们有一个batch_size为2，类别数为3的模型输出（probabilities）
# 和对应的软标签（probabilities）
batch_size = 1
num_classes = 3

# 随机生成模型输出（probabilities），确保每行的和为1（或接近1）
# model_outputs = torch.rand(batch_size, num_classes)
model_outputs = torch.tensor([[1, 3, 1]], dtype=torch.float32)
# model_outputs = F.softmax(model_outputs, dim=1)
print(model_outputs.softmax(dim=-1))
# 由于KLDivLoss期望输入是log-probabilities，我们需要对模型输出应用log变换
# log_model_outputs = F.log_softmax(model_outputs, dim=1)
log_model_outputs = F.log_softmax(model_outputs, dim=1)
print(log_model_outputs)

# 随机生成软标签（probabilities），确保每行的和为1（或接近1）
# soft_labels = torch.rand(batch_size, num_classes)
# soft_labels = F.softmax(soft_labels, dim=1)  # 确保软标签是有效的概率分布

# soft_labels = torch.randint(0, num_classes, (batch_size,))
# soft_labels = F.one_hot(soft_labels, num_classes).float()
soft_labels = torch.tensor([[0.0, 1.0, 0.0]], dtype=torch.float32)
print(soft_labels)

criterion = nn.KLDivLoss(reduction='batchmean', log_target=False)  # 注意：这里log_target应该设置为False，因为我们传递的是probabilities，
loss = criterion(log_model_outputs, soft_labels)
# 打印损失
print('KLDivLoss: ', loss.item())

criterion = nn.CrossEntropyLoss()
loss = criterion(log_model_outputs, soft_labels)
print('CrossEntropyLoss: ', loss.item())

tensor([[0.1065, 0.7870, 0.1065]])
tensor([[-2.2395, -0.2395, -2.2395]])
tensor([[0., 1., 0.]])
KLDivLoss:  0.2395448386669159
CrossEntropyLoss:  0.2395448386669159


In [1]:
# KL散度损失

import torch
import torch.nn as nn
import torch.nn.functional as F

# 假设我们有一个batch_size为2，类别数为3的模型输出（probabilities）
# 和对应的软标签（probabilities）
batch_size = 4
num_heads = 3
L = 128**2
num_classes = 256

# 模型输出
model_outputs = F.softmax(torch.randn((batch_size, num_heads, L, num_classes), dtype=torch.float32) * 10, dim=-1)
log_model_outputs = model_outputs.log()

# print(model_outputs)

# 随机生成软标签（probabilities），确保每行的和为1（或接近1）
# soft_labels = torch.randint(0, num_classes, (batch_size, num_heads, L))
# soft_labels = model_outputs.argmax(dim=-1)
# soft_labels_onehot = F.one_hot(soft_labels, num_classes).float()

soft_labels_onehot = model_outputs

# print(soft_labels)
# print(soft_labels_onehot)

# 计算KL散度损失
criterion = nn.KLDivLoss(reduction='batchmean', log_target=False)  # 注意：这里log_target应该设置为False，因为我们传递的是probabilities，
loss = criterion(log_model_outputs.flatten(0,-2), soft_labels_onehot.flatten(0,-2))
print('KLDivLoss: ', loss.item())
loss = criterion(log_model_outputs, soft_labels_onehot)
print('KLDivLoss: ', loss.item())

# # 计算交叉熵损失
# criterion = nn.CrossEntropyLoss()
# loss = criterion(model_outputs.flatten(0,-2), soft_labels_onehot.flatten(0,-2))
# print('CrossEntropyLoss: ', loss.item())
# loss = criterion(model_outputs.flatten(0,-2), soft_labels.flatten())
# print('CrossEntropyLoss: ', loss.item())


KLDivLoss:  -1.390041901028205e-11
KLDivLoss:  -6.832333951933833e-07


In [13]:
# softmax与log_softmax对比
import torch
import torch.nn as nn
import torch.nn.functional as F

a = torch.randn(3)
print(F.log_softmax(a, dim=-1))
print(F.softmax(a, dim=-1).log())

tensor([-0.6214, -1.0354, -2.2281])
tensor([-0.6214, -1.0354, -2.2281])


In [5]:
# 统计二维及以上维度矩阵某一行元素中各元素出现的频率
import torch

def getBlocks(x_2d: torch.Tensor, stride: int = 4):
    '''
    x_2d    : [batch_size, dim, height, width]
    stride  : 滑窗尺寸
    '''
    B, dim, H, W = x_2d.shape
    bs = stride ** 2                                                # block size
    nb = int(H*W / bs)                                              # num blocks
    x_unfold = F.unfold(                                            # [B, nb, bs, dim]
        x_2d.float(), 
        kernel_size=stride, 
        padding=0, 
        stride=stride
    ).view(B, dim, bs, nb).permute(0, 3, 2, 1).contiguous()         # [B, dim*bs, nb] -> [B, dim, bs, nb] -> [B, nb, bs, dim]
    return x_unfold

num_classes = 150
B = 8
dim = 1
H = 128
W = 128
stride = 8
L_ = H*W // stride**2

fm = torch.randint(0, num_classes, (B, dim, H, W))              # [B, dim, H, W]
# print(fm)

# input = torch.tensor([0, 1, 0, 1, 2, 2, 3, 3, 3, 149])
# input = torch.randint(0, num_classes, (B, L_, d))
input = getBlocks(fm, stride).squeeze().long()                  # (B, L_, bs)
# print(input)

index_add = torch.arange(0, B*L_).reshape(B, L_, 1) * num_classes
# print(input.shape)
# print(index_add.shape)

input_encode = input + index_add
input_encode_flatten = input_encode.flatten()
# print(input_encode)

counts = torch.bincount(input_encode_flatten)
padding = B * L_ * num_classes - counts.shape[0]
counts = torch.cat([counts, torch.zeros(padding, dtype=torch.long)], dim=0)    
# print(counts)

frequency = counts.reshape(B, L_, num_classes) / stride**2  # [B, L_, num_classes]
# print(frequency)

# [B, L, L_]
fm = fm.flatten(1)
results = []
for i in range(B):
    results.append(frequency[i][:, fm[i]].transpose(-2, -1))
results = torch.stack(results)
print(results.shape)
# print(results)


torch.Size([8, 16384, 256])


In [48]:
# affinity loss测试
import torch
import torch.nn as nn
import torch.nn.functional as F

def getMask(size: tuple, index_onehot: torch.Tensor, gain: float=1.0):
    '''
    size: 特征图尺寸, (h, w)
    index_onehot: 聚类结果(每个像素对应的聚类中心的one-hot索引), [B, num_heads, L, S]
    gain: 增益系数
    '''
    assert type(size) == tuple, 'Data type of size in function <getMask> should be <tuple>!'
    assert size.__len__() == 2, 'Length of size should be 2!'
    coords_h = torch.arange(size[0])
    coords_w = torch.arange(size[1])
    coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 构造坐标窗口元素坐标索引，[2, h, w]
    # 一维化特征图像素坐标，[2, L]
    coords_featuremap = torch.flatten(coords, start_dim=1).float().to(index_onehot.device)
    # [B, num_heads, 2, L]
    coords_featuremap = coords_featuremap.reshape(
        1, 1, 2, -1
    ).repeat(index_onehot.shape[0], index_onehot.shape[1], 1, 1)
    # [B, num_heads, 1, S]
    index_onehot_sum = torch.sum(index_onehot, dim=-2, keepdim=True)
    index_onehot_sum[index_onehot_sum==0] = 1
    
    # 聚类中心坐标，[B, num_heads, 2, S]
    coords_clustercenter = torch.einsum(
        'bhcl,bhls->bhcs', coords_featuremap, index_onehot
    ) / index_onehot_sum

    # 构造相对位置矩阵, 第一个矩阵是h方向的相对位置差, 第二个矩阵是w方向的相对位置差
    relative_coords = coords_featuremap[:, :, :, :, None] - coords_clustercenter[:, :, :, None, :]
    distance = torch.sqrt(                                      # [B, num_heads, L, S]
        torch.square(relative_coords[:,:,0,:,:]) + torch.square(relative_coords[:,:,1,:,:])
    )
    # exp操作用于处理distance中的0, [B, num_heads, L, S]
    distance_exp = torch.exp(distance)
    # 距离越远的token注意力增强越少(加性增强), 最大值为1*gain, 最小值可以接近0, [B, num_heads, L, S]
    mask = (1 / distance_exp) * gain
    return mask

def initRPE(shape_x, shape_c, window_size):
    '''
    输出：
    delta_onehot
    relative_position_bias_init
    '''
    batch_size, num_heads, L_, _ = shape_c
    _, _, H, W, _ = shape_x
    H_, W_ = H // window_size[0], W // window_size[1]
    N = window_size[0] * window_size[1]

    delta_index = torch.arange(L_).reshape(1, 1, L_, 1).repeat(batch_size, num_heads, 1, N)
    delta_index = delta_index.reshape(
        batch_size, num_heads, H_, W_, window_size[0], window_size[1]
    ).permute(
        0, 1, 2, 4, 3, 5
    ).reshape(
        batch_size, num_heads, H_ * window_size[0], W_ * window_size[1]
    ).reshape(
        batch_size, num_heads, H*W
    )                                                                   # [B, num_heads, L]
    delta_onehot = F.one_hot(delta_index, L_).float()                   # [B, num_heads, L, L']
    relative_position_bias_init = getMask((H, W), delta_onehot)    # [B, num_heads, L, L']
    return delta_onehot, relative_position_bias_init

def initCenter(x: torch.Tensor, stride: int):
    '''
    输入: 
    x: tensor, [batch_size, num_heads, H, W, head_embed_dims], 待聚类数据
    输出:
    c_init: tensor, [batch_size, num_heads, L', head_embed_dims], 聚类中心
    '''
    batch_size, num_heads, H, W, head_embed_dims = x.shape

    # [batch_size*num_heads, head_embed_dims, H, W]
    x_temp = x.permute(
        0, 1, 4, 2, 3
    ).reshape(
        batch_size*num_heads, head_embed_dims, H, W
    )
    # [batch_size, num_heads, L', head_embed_dims]
    c_init = F.interpolate(x_temp, scale_factor=1/stride, mode='nearest').reshape(
        batch_size, num_heads, head_embed_dims, -1
    ).transpose(-2, -1)
    # 单位化c_init, 便于后续进行余弦相似度计算
    c_init = F.normalize(c_init, dim=-1)
    return c_init

def updateCenter(x: torch.Tensor, c: torch.Tensor, relative_position_bias: None):
    '''
    输入: 
    x: tensor, [batch_size, num_heads, H, W, head_embed_dims], 待聚类数据
    c: tensor, [batch_size, num_heads, L', head_embed_dims], 初始化聚类中心
    relative_position_bias: tensor, [batch_size, num_heads, L, L'], 相对位置编码
    输出：
    c_new: tensor, [B, num_heads, L', head_embed_dims], 更新后的聚类中心
    affinity: 
    '''
    batch_size, num_heads, H, W, head_embed_dims = x.shape
    _, _, L_, _ = c.shape

    x = x.reshape(batch_size, num_heads, -1, head_embed_dims)

    # [B, num_heads, L, L']
    scale = torch.tensor(15.0)
    affinity_raw = torch.einsum('bhld,bhmd->bhlm', x, c)
    affinity = affinity_raw  * scale
    if not relative_position_bias is None:
        affinity = affinity + relative_position_bias
    
    affinity_aux = affinity_raw * scale.detach()
    if not relative_position_bias is None:
        affinity_aux = affinity_aux + relative_position_bias

    # 使用掩膜进行非极大值抑制
    affinity_mask = torch.zeros_like(affinity)
    affinity_mask[affinity < affinity.max(dim=-1, keepdim=True)[0]] = -torch.inf
    # [B, num_heads, L, L']
    affinity_onehot = torch.softmax(affinity + affinity_mask, dim=-1)

    # 更新聚类中心
    # [B, num_heads, L', head_embed_dims]
    c_sum = torch.einsum('bhlm,bhld->bhmd', affinity_onehot, x)
    # 直接单位化，就不用affinity归一化了
    c_new = F.normalize(c_sum, dim=-1)

    return c_new, affinity_onehot, affinity_aux, scale.detach()

def getAffinity_label(labels: torch.Tensor, stride: int = 4, num_classes: int = 150):
    '''
    labels: [B, 1, H, W]
    stride: 下采样倍率
    num_classes: 分割类别数
    '''
    B, dim, H, W = labels.shape
    assert dim == 1

    # 分块，模拟下采样的过程，更加精细
    bs = stride ** 2                                    # block size
    nb = int(H*W / bs)                                  # num blocks
    labels_unfold = F.unfold(                           # [B, nb, bs, dim]
        labels.float(), 
        kernel_size=stride, 
        padding=0, 
        stride=stride
    ).reshape(B, bs, nb).transpose(-2, -1).long()       # [B, dim*bs, nb] -> [B, nb, bs]

    # 用于区分聚类中心每一行的编码
    index_add = torch.arange(0, B*nb, device=labels.device).reshape(B, nb, 1) * num_classes
    labels_unfold_encode = labels_unfold + index_add

    # 统计聚类中心的类别成分占比
    labels_unfold_encode_flatten = labels_unfold_encode.flatten()
    counts = torch.bincount(labels_unfold_encode_flatten)
    padding = B * nb * num_classes - counts.shape[0]    # 最后一行特殊处理
    counts = torch.cat([counts, torch.zeros(padding, dtype=torch.long, device=labels.device)], dim=0)
    frequency = counts.reshape(B, nb, num_classes) / bs # [B, nb, num_classes]

    # 使用labels索引从聚类中心占比中索引出一个亲和力矩阵，亲和力矩阵的每个元素为labels元素在聚类中心分块中同类元素的占比
    labels = labels.flatten(1).long()                   # [B, L]
    affinity = []
    for i in range(B):
        affinity.append(frequency[i][:, labels[i]].transpose(-2, -1))
    affinity = torch.stack(affinity).unsqueeze(1)       # [B, 1, L, nb]
    return affinity

# 1. 构造原始数据: [batch_size, num_heads, H, W, head_embed_dims]
batch_size = 1
num_heads = 1
H, W = 4, 4
stride = 2
H_, W_ = H // stride, W // stride
L_ = H_ * W_
head_embed_dims = 4

x_label = torch.randint(0, head_embed_dims, (batch_size, 1, H_, W_, 1))  # [batch_size, 1, H, W]
x_label = x_label.repeat(
    1, 1, 1, 1, stride**2
).reshape(
    batch_size, 1, H_, W_, stride, stride
).permute(
    0, 1, 2, 4, 3, 5
).reshape(
    batch_size, 1, H, W
)
# print('x_label: {}\n{}'.format(x_label.shape, x_label))

x_feature = F.one_hot(x_label.repeat(1, num_heads, 1, 1), head_embed_dims).float()
# print('x_feature: {}\n{}'.format(x_feature.shape, x_feature))

# 2. 计算affinity_aux
c = initCenter(x_feature, stride)   # [batch_size, num_heads, head_embed_dims, L_]
# print(c)
_, relative_position_bias = initRPE(x_feature.shape, c.shape, (stride, stride))
# print('relative_position_bias: {}\n{}'.format(relative_position_bias.shape, relative_position_bias))
# _, _, affinity_aux, scale = updateCenter(x_feature, c, relative_position_bias=None)       # 无相对位置编码
# print('affinity_aux_nopoe: {}\n{}'.format(affinity_aux.shape, affinity_aux))
_, _, affinity_aux, scale = updateCenter(x_feature, c, relative_position_bias)              # 有相对位置编码
# print('affinity_aux: {}\n{}'.format(affinity_aux.shape, affinity_aux))

# 3. 构造affinity_label
affinity_label = getAffinity_label(x_label, stride, head_embed_dims+1) * scale + relative_position_bias     # [batch_size, num_heads, L, L']
# print('affinity_label: {}\n{}'.format(affinity_label.shape, affinity_label))

# 4. 计算affinity loss
affinity_log_softmax = F.log_softmax(affinity_aux, dim=-1)  # [batch_size, num_heads, L, L']
affinity_label_softmax = F.softmax(affinity_label, dim=-1)  # [batch_size, num_heads, L, L']

# print(affinity_log_softmax.shape)
# print(affinity_label_softmax.shape)

KLDivLoss = nn.KLDivLoss(reduction='batchmean', log_target=False)
loss = KLDivLoss(affinity_log_softmax.flatten(0,-2), affinity_label_softmax.flatten(0,-2))
print('affinity loss: {:.6f}'.format(loss))



affinity loss: 0.000000
